<a href="https://colab.research.google.com/github/10100111/Display-of-HW1/blob/main/7_Pro_%D0%9F%D0%BE%D0%BB%D0%BD%D0%BE%D1%81%D0%B2%D1%8F%D0%B7%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8%2C_%D0%BE%D0%B1%D1%83%D1%87%D0%B0%D1%8E%D1%89%D0%B0%D1%8F_%D0%B8_%D1%82%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%B0%D1%8F_%D0%B2%D1%8B%D0%B1%D0%BE%D1%80%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание Pro

Макс 10 баллов
Повысьте точность модели по обнаружению мин (база sonar)  до 90 % на тестовой выборке. 

Можно использовать различные варианты слоев Dropout и BatchNormalization. Можно менять количество примеров в обучающей и проверочной выборках, но нельзя менять количество примеров в тестовой.

In [ ]:
from tensorflow.keras.models import Sequential # НС прямого распространения
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization # Основные слои
from tensorflow.keras import utils # Утилиты для to_categorical
from tensorflow.keras.preprocessing import image # Для отрисовки изображения
from tensorflow.keras.optimizers import Adam, Adadelta # Алгоритмы оптимизации, для настройки скорости обучения
import numpy as np # Библиотека работы с массивами
import matplotlib.pyplot as plt # Отрисовка изображений
from PIL import Image # Отрисовка изображений
import pandas as pd # Библиотека pandas
from google.colab import files # Импорт файлов
from sklearn.preprocessing import LabelEncoder, StandardScaler # Функции для нормализации данных
from sklearn import preprocessing # Пакет предварительной обработки данных

# Отрисовывать изображения в ноутбуке, а не в консоль или файл
%matplotlib inline


In [ ]:
# Загружаем файлы
files.upload()
!ls # Выводим содержимое корневой папки

Saving sonar.csv to sonar.csv
sample_data  sonar.csv


In [ ]:
# Загружаем данные из файла sonar.csv
df = pd.read_csv("sonar.csv", header=None)   # header=None, когда данные не имеют строки с заголовками
df.head()                                    # Выводим первые 5 строк наших данных

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [ ]:
print(df.shape) # Размерность данных

(208, 61)


In [ ]:
dataset = df.values                 # Берем только значения массива(без индексов)
X = dataset[:,0:60].astype(float)   # Присваиваем им тип данных - float(с плавающей точкой) данным с 0 по 60 колонки
Y = dataset[:,60]                   # Присваеваем значению Y данные из столбца с индексом 60
Y[Y=='R']='0'                       # Если значение элемента столбца равно 'R', присваеваем ему значение '0'
Y[Y=='M']='1'                       # Если значение элемента столбца равно 'Y', присваеваем ему значение '1'
Y = Y.astype(int)                   # Меняем тип данных столбца на 'int'(целочисленный тип данных)
print(X.shape)                      # Выводим размерность X
print(Y.shape)                      # Выводим размерность Y
print(Y)                            

(208, 60)
(208,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
# Создание обучающей, проверочной и тестовой выборки
# sklearn - популярная библиотека для машинного обучения
# train_test_split - функция разделения на обучающую и проверочную/тестовую выборку
from sklearn.model_selection import train_test_split


# test_size=0.2 - будет выделено 20% от тренировочных данных 
# shuffle=True - перемешать данные
# x_train - данные для обучения
# x_test - данные для проверки
# y_train - правильные ответы для обучения
# y_test - правильные ответы для проверки
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)    
print (x_train.shape)
print (x_test.shape)
print (y_train.shape)
print (y_test.shape)

(166, 60)
(42, 60)
(166,)
(42,)


In [ ]:
# Функция пересоздаёт пустую сеть
def createModel():

  # Создаем сеть
  model = Sequential()

  # Добавляем слои
  model.add(Dense(60, input_dim=60, activation='relu'))
  model.add(Dense(30,  activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  # Компилируем сеть
  model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

  return model # Возвращаем созданную сеть

In [ ]:
# Сделаем списки с количеством примеров  n_val - проверочная выборка и count_list - обучающая выборка
x_len = x_train.shape[0] # Запоминаем размер всей выборки целиком
n_val = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
count_list = [x_len-x for x in n_val]
print(count_list)

[161, 156, 151, 146, 141, 136, 131, 126, 121, 116, 111, 106, 101, 96, 91, 86]


In [ ]:
# Запускаем в цикле создание и обучение нейронки на обучающем наборе данных из примеров в количестве от 86 до 161 с шагом 5, 
# чтобы понять какая выборка оптимальна для большей точности

for i in range(len(n_val)):
  print('Обучающая выборка', count_list[i], 'примеров')

  model = createModel()

  # Обучаем сеть
  model.fit(x_train[:x_len-n_val[i]], 
            y_train[:x_len-n_val[i]], 
            batch_size=8, 
            epochs=100,            
            validation_data=(x_train[x_len-n_val[i]:], y_train[x_len-n_val[i]:]), 
            verbose=0)
    # Вычисляем результаты сети на тестовом наборе
  scores = model.evaluate(x_test, y_test, verbose=1)

  # scores состоит из двух знанчений
  # scores[0] - loss сети на тестовой выборке
  # scores[1] - процент правильно распознанных примеров на тестовой выборке
  print(scores)
  print("Доля верных ответов на тестовых данных, в процентах: ", round(scores[1] * 100, 4), "%", sep="")
  prediction = model.predict(x_test) # Делаем предикт тестовой выборки
  n = np.random.randint(0, 42)        # Предиктим рандомное число из базы
  print('Предсказание ', np.argmax(prediction[n]))
  print('Правильный ответ', y_test[n])

Обучающая выборка 161 примеров
2/2 [==============================] - 0s 8ms/step - loss: 0.3852 - accuracy: 0.9048
[0.38518092036247253, 0.9047619104385376]
Доля верных ответов на тестовых данных, в процентах: 90.4762%
Предсказание  0
Правильный ответ 0
Обучающая выборка 156 примеров
2/2 [==============================] - 0s 10ms/step - loss: 0.4136 - accuracy: 0.8571
[0.41360577940940857, 0.8571428656578064]
Доля верных ответов на тестовых данных, в процентах: 85.7143%
Предсказание  0
Правильный ответ 0
Обучающая выборка 151 примеров
2/2 [==============================] - 0s 10ms/step - loss: 0.4421 - accuracy: 0.8571
[0.4420541524887085, 0.8571428656578064]
Доля верных ответов на тестовых данных, в процентах: 85.7143%
Предсказание  0
Правильный ответ 0
Обучающая выборка 146 примеров
2/2 [==============================] - 0s 8ms/step - loss: 0.4798 - accuracy: 0.8333
[0.47975650429725647, 0.8333333134651184]
Доля верных ответов на тестовых данных, в процентах: 83.3333%
Предсказание  

In [ ]:
# Путем нескольких запусков определились оптимальные значения обучающего набора данных от 86 до 111 примеров, а также 161 пример
# Достигнут показатель доли верных ответов на тестовых данных, в процентах: 90.4762% при обучающей выборке 161 пример
# К текущему эксперименту с выборкой применим слои Dropout и BatchNormalization 
# Создадим новую модель сети

def createModelDropoutBatchNormalization():

  # Создаем сеть
  model = Sequential()

  # Добавляем слои
  # model.add(BatchNormalization()) 
  model.add(Dropout(0.2, input_dim=60))
  model.add(Dense(60, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))
  model.add(Dense(30, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))

  # Компилируем сеть
  model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

  return model # Возвращаем созданную сеть

In [ ]:

for i in range(len(n_val)):
  print('Обучающая выборка', count_list[i], 'примеров')

  model = createModelDropoutBatchNormalization()

  # Обучаем сеть
  model.fit(x_train[:x_len-n_val[i]], 
            y_train[:x_len-n_val[i]], 
            batch_size=8, 
            epochs=100,            
            validation_data=(x_train[x_len-n_val[i]:], y_train[x_len-n_val[i]:]), 
            verbose=0)
    # Вычисляем результаты сети на тестовом наборе
  scores = model.evaluate(x_test, y_test, verbose=1)

  # scores состоит из двух знанчений
  # scores[0] - loss сети на тестовой выборке
  # scores[1] - процент правильно распознанных примеров на тестовой выборке
  print(scores)
  print("Доля верных ответов на тестовых данных, в процентах: ", round(scores[1] * 100, 4), "%", sep="")
  prediction = model.predict(x_test) # Делаем предикт тестовой выборки
  n = np.random.randint(0, 42)        # Предиктим рандомное число из базы
  print('Предсказание ', np.argmax(prediction[n]))
  print('Правильный ответ', y_test[n])

Обучающая выборка 161 примеров
2/2 [==============================] - 0s 10ms/step - loss: 0.2440 - accuracy: 0.8810
[0.24397200345993042, 0.8809523582458496]
Доля верных ответов на тестовых данных, в процентах: 88.0952%
Предсказание  0
Правильный ответ 0
Обучающая выборка 156 примеров
2/2 [==============================] - 0s 11ms/step - loss: 0.3239 - accuracy: 0.8810
[0.3239283859729767, 0.8809523582458496]
Доля верных ответов на тестовых данных, в процентах: 88.0952%
Предсказание  0
Правильный ответ 1
Обучающая выборка 151 примеров
2/2 [==============================] - 0s 10ms/step - loss: 0.2807 - accuracy: 0.8810
[0.28071328997612, 0.8809523582458496]
Доля верных ответов на тестовых данных, в процентах: 88.0952%
Предсказание  0
Правильный ответ 1
Обучающая выборка 146 примеров
2/2 [==============================] - 0s 9ms/step - loss: 0.3313 - accuracy: 0.8571
[0.33125340938568115, 0.8571428656578064]
Доля верных ответов на тестовых данных, в процентах: 85.7143%
Предсказание  0


In [ ]:
# Несколько запусков сети со слоями Dropout и BatchNormalization в разных комбинациях дал 90.4762% на тестовой выборке ,при обучающей выборке из 91 примера
# Создадим новую модель сети без слоя Normalization

def createModelDropout():

  # Создаем сеть
  model = Sequential()

  # Добавляем слои
  
  model.add(Dropout(0.2, input_dim=60))
  model.add(Dense(60, activation='relu'))
  
  model.add(Dropout(0.2))
  model.add(Dense(30, activation='relu'))
  
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))

  # Компилируем сеть
  model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

  return model # Возвращаем созданную сеть

In [ ]:
# Запускаем обучение сети без слоя нормализации c Dropout слоем и получаем точность 90.4762% на тестовой выборке ,при обучающей выборке из 151 или 91 примеров

for i in range(len(n_val)):
  print('Обучающая выборка', count_list[i], 'примеров')

  model = createModelDropout()

  # Обучаем сеть
  model.fit(x_train[:x_len-n_val[i]], 
            y_train[:x_len-n_val[i]], 
            batch_size=8, 
            epochs=100,            
            validation_data=(x_train[x_len-n_val[i]:], y_train[x_len-n_val[i]:]), 
            verbose=0)
    # Вычисляем результаты сети на тестовом наборе
  scores = model.evaluate(x_test, y_test, verbose=1)

  # scores состоит из двух знанчений
  # scores[0] - loss сети на тестовой выборке
  # scores[1] - процент правильно распознанных примеров на тестовой выборке
  print(scores)
  print("Доля верных ответов на тестовых данных, в процентах: ", round(scores[1] * 100, 4), "%", sep="")
  prediction = model.predict(x_test) # Делаем предикт тестовой выборки
  n = np.random.randint(0, 42)        # Предиктим рандомное число из базы
  print('Предсказание ', np.argmax(prediction[n]))
  print('Правильный ответ', y_test[n])

Обучающая выборка 161 примеров
2/2 [==============================] - 0s 11ms/step - loss: 0.3131 - accuracy: 0.8810
[0.3131074607372284, 0.8809523582458496]
Доля верных ответов на тестовых данных, в процентах: 88.0952%
Предсказание  0
Правильный ответ 0
Обучающая выборка 156 примеров
2/2 [==============================] - 0s 10ms/step - loss: 0.2864 - accuracy: 0.8810
[0.2863517999649048, 0.8809523582458496]
Доля верных ответов на тестовых данных, в процентах: 88.0952%
Предсказание  0
Правильный ответ 0
Обучающая выборка 151 примеров
2/2 [==============================] - 0s 11ms/step - loss: 0.3240 - accuracy: 0.9048
[0.32404083013534546, 0.9047619104385376]
Доля верных ответов на тестовых данных, в процентах: 90.4762%
Предсказание  0
Правильный ответ 0
Обучающая выборка 146 примеров
2/2 [==============================] - 0s 11ms/step - loss: 0.3278 - accuracy: 0.8333
[0.3277680277824402, 0.8333333134651184]
Доля верных ответов на тестовых данных, в процентах: 83.3333%
Предсказание  

In [ ]:
# Создадим новую модель сети без слоя Dropout со слоем нормализации, интересно посмотреть что будет ))

def createModelBatchNormalization():

  # Создаем сеть
  model = Sequential()

  # Добавляем слои BatchNormalization вместо Dropout
  
  model.add(BatchNormalization())
  model.add(Dense(60, activation='relu'))
  
  model.add(BatchNormalization())
  model.add(Dense(30, activation='relu'))
  
  model.add(BatchNormalization())
  model.add(Dense(1, activation='sigmoid'))

  # Компилируем сеть
  model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

  return model # Возвращаем созданную сеть

In [ ]:
# Запускаем обучение сети без слоя Dropout и получаем:
# - точность 92.8571% на тестовой выборке ,при обучающей выборке из 141 , 136, 106 примеров
# - точность 90.4762% на тестовой выборке ,при обучающей выборке из 126 примеров

for i in range(len(n_val)):
  print('Обучающая выборка', count_list[i], 'примеров')

  model = createModelBatchNormalization()

  # Обучаем сеть
  model.fit(x_train[:x_len-n_val[i]], 
            y_train[:x_len-n_val[i]], 
            batch_size=8, 
            epochs=100,            
            validation_data=(x_train[x_len-n_val[i]:], y_train[x_len-n_val[i]:]), 
            verbose=0)
    # Вычисляем результаты сети на тестовом наборе
  scores = model.evaluate(x_test, y_test, verbose=1)

  # scores состоит из двух знанчений
  # scores[0] - loss сети на тестовой выборке
  # scores[1] - процент правильно распознанных примеров на тестовой выборке
  print(scores)
  print("Доля верных ответов на тестовых данных, в процентах: ", round(scores[1] * 100, 4), "%", sep="")
  prediction = model.predict(x_test) # Делаем предикт тестовой выборки
  n = np.random.randint(0, 42)        # Предиктим рандомное число из базы
  print('Предсказание ', np.argmax(prediction[n]))
  print('Правильный ответ', y_test[n])

Обучающая выборка 161 примеров
2/2 [==============================] - 0s 8ms/step - loss: 0.3851 - accuracy: 0.8333
[0.385055273771286, 0.8333333134651184]
Доля верных ответов на тестовых данных, в процентах: 83.3333%
Предсказание  0
Правильный ответ 1
Обучающая выборка 156 примеров
2/2 [==============================] - 0s 6ms/step - loss: 0.4996 - accuracy: 0.8571
[0.49959826469421387, 0.8571428656578064]
Доля верных ответов на тестовых данных, в процентах: 85.7143%
Предсказание  0
Правильный ответ 1
Обучающая выборка 151 примеров
2/2 [==============================] - 0s 12ms/step - loss: 0.3954 - accuracy: 0.8810
[0.3954109847545624, 0.8809523582458496]
Доля верных ответов на тестовых данных, в процентах: 88.0952%
Предсказание  0
Правильный ответ 0
Обучающая выборка 146 примеров
2/2 [==============================] - 0s 10ms/step - loss: 0.4055 - accuracy: 0.8810
[0.40548768639564514, 0.8809523582458496]
Доля верных ответов на тестовых данных, в процентах: 88.0952%
Предсказание  0


Резюме:
Максимальная точность 92.8571% достигнута на тестовой выборке ,при обучающей выборке из 141 , 136, 106 примеров с применением слоев BatchNormalization